In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
/home/codespace/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

# As the data is huge. Taking only 1% of the overall data.
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet').sample(frac=0.01, random_state=42)


In [11]:
df_val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration,PU_DO
2004046,2,2023-02-20 19:29:12,2023-02-20 19:48:51,1.0,4.31,1.0,N,140,68,1,...,0.0,0.5,0.00,0.00,1.0,26.60,2.5,0.00,19.650000,140_68
332043,2,2023-02-04 01:46:13,2023-02-04 02:05:51,1.0,8.69,1.0,N,138,230,2,...,6.0,0.5,0.00,6.55,1.0,54.40,2.5,1.25,19.633333,138_230
2635502,2,2023-02-26 19:08:53,2023-02-26 19:15:39,1.0,3.04,1.0,N,132,10,2,...,0.0,0.5,0.00,0.00,1.0,16.25,0.0,1.25,6.766667,132_10
2474057,2,2023-02-25 10:21:38,2023-02-25 10:36:35,1.0,3.16,1.0,N,239,152,1,...,0.0,0.5,5.00,0.00,1.0,26.70,2.5,0.00,14.950000,239_152
547538,1,2023-02-06 15:29:14,2023-02-06 15:52:31,1.0,2.90,1.0,N,237,186,1,...,2.5,0.5,4.45,0.00,1.0,26.85,2.5,0.00,23.283333,237_186


In [9]:
print(f"Train shape:{df_train.shape}")
print(f"Test shape:{df_val.shape}")

Train shape:(3009173, 20)
Test shape:(28560, 20)


In [20]:
df_train[(df_train.duration >= 1) & (df_train.duration <= 60)].shape[0]
df_val[(df_val.duration >= 1) & (df_val.duration <= 60)].shape[0]

28560

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
df_train.describe().round(2).T

,count,mean,min,25%,50%,75%,max,std
VendorID,3009173.0,1.73,1.0,1.0,2.0,2.0,2.0,0.44
tpep_pickup_datetime,3009173,2023-01-17 00:34:19.090805,2022-10-25 00:42:10,2023-01-09 16:26:19,2023-01-17 08:45:30,2023-01-24 16:33:29,2023-02-01 00:56:53,NaN
tpep_dropoff_datetime,3009173,2023-01-17 00:48:31.382655,2022-10-25 00:44:22,2023-01-09 16:40:24,2023-01-17 08:59:46,2023-01-24 16:48:53,2023-02-01 01:06:43,NaN
passenger_count,2938068.0,1.36,0.0,1.0,1.0,1.0,8.0,0.9
trip_distance,3009173.0,3.78,0.0,1.09,1.8,3.3,258928.15,251.96
RatecodeID,2938068.0,1.42,1.0,1.0,1.0,1.0,99.0,6.05
payment_type,3009173.0,1.18,0.0,1.0,1.0,1.0,4.0,0.51
fare_amount,3009173.0,17.89,-580.0,8.6,12.8,19.8,999.0,16.21
extra,3009173.0,1.55,-7.5,0.0,1.0,2.5,12.5,1.78
mta_tax,3009173.0,0.49,-0.5,0.5,0.5,0.5,53.16,0.09


In [13]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [14]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [15]:
X_train[:5].toarray().shape

(5, 21802)

In [16]:
X_train[:5].toarray()

array([[0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.97],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 1.1 ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 2.51],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 1.9 ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 1.43]])

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

# root_mean_squared_error(y_val, y_pred)
mean_squared_error(y_val, y_pred, squared=False)

5.170621949347703

In [18]:
with open('models/hw_lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [19]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

# root_mean_squared_error(y_val, y_pred)
mean_squared_error(y_val, y_pred, squared=False)

9.389979260684303